In [16]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import random


In [17]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\diego.guizanlopez\\.wdm\\drivers\\geckodriver\\win64\\v0.34.0\\geckodriver.exe'

In [18]:
#Instalar gecko
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service


#Abrir un navegador
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Firefox()

In [19]:
URL="https://technical.city/en/video/rating"

In [20]:

def waitFinishLoad(driver,idLoader,needsDisapeared=False,by=By.ID,needsWait=True):
    """Espera a la carga de un elemento antes de continuar

    Args:
        driver (WebDriver): WebDriver
        idLoader (str): Id a buscar, puede ser XPath...
        needsDisapeared (bool, optional): Si el elemento a esperar tiene que aparecer o desaparecer como un CircularProgress. Defaults to False.
        by (str, optional): Tipo de By a buscar el Wait. Defaults to By.ID.
        needsWait (bool, optional): Si necesita esperar un extra para que no pueda saltar un Captcha. Defaults to False.
    """
    if idLoader is None:
        return
    WebDriverWait(driver, timeout=60).until(EC.presence_of_element_located((by, idLoader)))
    if needsDisapeared:
        WebDriverWait(driver, timeout=60).until(EC.invisibility_of_element((by, idLoader)))
    if needsWait:
        time.sleep(random.choice([0.05,0.1,0.2]))

def elementBy(query,driver,**kargs):
    """Devuelve un element|None dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        Element|None: Elemento de la búsqueda
    """
    idLoader=kargs.get("idLoader",query)
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,idLoader,by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_element(kargs.get("by",By.XPATH),query)
    waitFinishLoad(driver,idLoader,by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element

def clickEvent(driver,element,**kargs):
    """Click en un elemento que funciona con evento

    Args:
        driver (WebDriver): WebDriver
        element (Element): Elemento que tiene el evento
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    if kargs.get("normal",False):
        element.click
    else:
        try:
            driver.execute_script(element.get_dom_attribute("onClick"))
        except:
            element.click
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))

def elementsBy(query,driver,**kargs):
    """Devuelve una lista de elements dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        List<Element>: Lista de elementos
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_elements(kargs.get("byq",By.XPATH),query)
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element
# endregion

In [21]:
driver.get(URL)

In [ ]:
elementBy("//*[@role='dialog']/div[2]/div/button[2]",driver,by=By.XPATH,needsWait=False).click()

In [26]:
# id-footer
from io import StringIO
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

column_names=['Graphics card','Type','Performance','Architecture','Year','Price Now','TDP']

df = pd.DataFrame([])
pastElements=[]
laps=0
while True:
    elements=elementsBy("//tr",driver,by=By.XPATH)
    driver.execute_script("arguments[0].scrollIntoView();", elements[-1])
    if pastElements==elements:
        laps+=1
    else:
        laps=0
    if laps==100:
        break
    pastElements=elements


#soup=BeautifulSoup(driver.page_source,'html.parser')
#df=pd.read_html(StringIO(str(soup.findAll("table")[0])))[0]

soup=BeautifulSoup(driver.page_source,'html.parser')
table=soup.findAll("table")[0]
# Eliminar los números de ranking
rows = table.find_all('tr')
rowsL=[]
for row in rows:
    cells = row.find_all('td')
    cellValues=[]
    for index,cell in enumerate(cells):
        if index==0:continue
        if index==1:
            try:
                cellValues.append(cell.find_all('img')[0]['title'].strip())
            except:
                cellValues.append(np.nan)
            continue
        cellValues.append(cell.text.strip())
    rowsL.append(cellValues)
    #rowsL.append(cellValues)
df = pd.DataFrame(columns=column_names,data=rowsL)
#df=df[~df.iloc[:,0].str.startswith('Página')]

In [28]:
df['none_count'] = df.apply(lambda x:x.isnull().sum(), axis=1)

# Elimina las filas que tienen más de 3 valores None
df = df[df['none_count'] <= 3]

df.drop(columns=["none_count"],inplace=True)


C:\Users\diego.guizanlopez\AppData\Local\Temp\ipykernel_14496\3810561771.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["none_count"],inplace=True)


In [30]:
df

,Graphics card,Type,Performance,Architecture,Year,Price Now,TDP
1,NVIDIA GeForce RTX 4090,desktop,100.00,Ada Lovelace,2022,1756 USD,450 W
2,NVIDIA GeForce RTX 4080,desktop,89.40,Ada Lovelace,2022,1451 USD,320 W
3,NVIDIA GeForce RTX 4080 SUPER,desktop,88.40,Ada Lovelace,2024,−,320 W
4,NVIDIA GeForce RTX 4070 Ti SUPER,desktop,82.03,Ada Lovelace,2024,−,285 W
5,NVIDIA GeForce RTX 4070 Ti,desktop,81.99,Ada Lovelace,2023,837 USD,285 W
...,...,...,...,...,...,...,...
1447,NVIDIA GeForce2 Ultra,desktop,−,Celsius,2000,−,−
1448,NVIDIA GeForce4 Ti 4400,desktop,−,Kelvin,2002,−,−
1449,NVIDIA GeForce4 Ti 4800 SE,desktop,−,Kelvin,2003,−,−
1450,NVIDIA Quadro NVS 280 PCI,workstation,−,Rankine,2003,100 USD,13 W


In [31]:
if df.columns.tolist()[0].startswith('@media'):
    # Si es así, elimina la columna
    df = df.drop(columns=[df.columns.tolist()[0]])

In [32]:
df=df[df.isna().sum(axis=1)<3]

In [33]:
len(df)

1446

In [34]:
df=df.reset_index()

In [35]:
df=df.drop("index",axis=1)
df

,Graphics card,Type,Performance,Architecture,Year,Price Now,TDP
0,NVIDIA GeForce RTX 4090,desktop,100.00,Ada Lovelace,2022,1756 USD,450 W
1,NVIDIA GeForce RTX 4080,desktop,89.40,Ada Lovelace,2022,1451 USD,320 W
2,NVIDIA GeForce RTX 4080 SUPER,desktop,88.40,Ada Lovelace,2024,−,320 W
3,NVIDIA GeForce RTX 4070 Ti SUPER,desktop,82.03,Ada Lovelace,2024,−,285 W
4,NVIDIA GeForce RTX 4070 Ti,desktop,81.99,Ada Lovelace,2023,837 USD,285 W
...,...,...,...,...,...,...,...
1441,NVIDIA GeForce2 Ultra,desktop,−,Celsius,2000,−,−
1442,NVIDIA GeForce4 Ti 4400,desktop,−,Kelvin,2002,−,−
1443,NVIDIA GeForce4 Ti 4800 SE,desktop,−,Kelvin,2003,−,−
1444,NVIDIA Quadro NVS 280 PCI,workstation,−,Rankine,2003,100 USD,13 W


In [37]:
import os
import pathlib


df.to_csv(os.path.join(str(pathlib.Path().resolve()),"GPU.csv"))